# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""55797038850086…",0.983033,"""16948537663694…","""../../../../da…","""10472646288361…","""64700685745251…",false
"""76180145749875…",0.144221,"""46744576257913…","""../../../../da…","""10472646288361…","""67443535116509…",true
"""16707077147951…",0.679745,"""15494964011706…","""../../../../da…","""63692263730449…","""64700685745251…",false
"""76180145749875…",1.0,"""16085867942915…","""../../../../da…","""12570257287852…","""64700685745251…",false
"""55797038850086…",0.718982,"""99649367074000…","""../../../../da…","""12570257287852…","""10918435678164…",true
"""16707077147951…",0.0,"""21793080343640…","""../../../../da…","""14672847984252…",null,false
"""16707077147951…",0.02333,"""10431147961915…","""../../../../da…","""12570257287852…","""10918435678164…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""92009474510078…",0.983033,"""61478662605825…","""../../../../da…","""FirstMortgage3…","""76733479736465…","""Rejected"""
"""17417832150163…",0.144221,"""83873106972024…","""../../../../da…","""FirstMortgage3…","""30089697893777…","""Accepted"""
"""15939123424539…",0.679745,"""15232212195729…","""../../../../da…","""MoneyMarketSav…","""76733479736465…","""Rejected"""
"""17417832150163…",1.0,"""53188687411172…","""../../../../da…","""BasicChecking""","""76733479736465…","""Rejected"""
"""92009474510078…",0.718982,"""13608370568780…","""../../../../da…","""BasicChecking""","""56249806083362…","""Accepted"""
"""15939123424539…",0.0,"""17248459100364…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""15939123424539…",0.02333,"""16906785844791…","""../../../../da…","""BasicChecking""","""56249806083362…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""16898686351729…",0.983033,"""12962370496612…","""../../../../da…","""FirstMortgage3…","""10090478402610…","""Rejected"""
"""17580231914390…",0.144221,"""16293076076726…","""../../../../da…","""FirstMortgage3…","""17064631154810…","""Accepted"""
"""75637516206865…",0.679745,"""22022067995138…","""../../../../da…","""MoneyMarketSav…","""10090478402610…","""Rejected"""
"""17580231914390…",1.0,"""26579696361435…","""../../../../da…","""BasicChecking""","""10090478402610…","""Rejected"""
"""16898686351729…",0.718982,"""10558355415811…","""../../../../da…","""BasicChecking""","""89594328646187…","""Accepted"""
"""75637516206865…",0.0,"""17592461772686…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""75637516206865…",0.02333,"""17600639351631…","""../../../../da…","""BasicChecking""","""89594328646187…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""13537465391195…",0.983033,"""13754077045841…","""FirstMortgage3…","""11041311262481…","""Rejected"""
"""14187909125814…",0.144221,"""55006337678146…","""FirstMortgage3…","""64827880101771…","""Accepted"""
"""37120664654019…",0.679745,"""43268515447312…","""MoneyMarketSav…","""11041311262481…","""Rejected"""
"""14187909125814…",1.0,"""11169118038290…","""BasicChecking""","""11041311262481…","""Rejected"""
"""13537465391195…",0.718982,"""18027760420142…","""BasicChecking""","""42190129228180…","""Accepted"""
"""37120664654019…",0.0,"""77087924571461…","""UPlusFinPerson…",null,"""Rejected"""
"""37120664654019…",0.02333,"""16068892626764…","""BasicChecking""","""42190129228180…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""af486bc1815c72…",0.983033,"""c73023dda51de1…","""../../../../da…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""3cdf7b1ae1342e…",0.144221,"""5414b197fb87b3…","""../../../../da…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""806ef34eb8237f…",0.679745,"""892f5d61a9215e…","""../../../../da…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""3cdf7b1ae1342e…",1.0,"""c3aa807d9e927a…","""../../../../da…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""af486bc1815c72…",0.718982,"""c2d492f9a05073…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""806ef34eb8237f…",0.0,"""b21059221605c4…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""806ef34eb8237f…",0.02333,"""80cbe489dcecd4…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
